In [1]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import requests

%matplotlib inline
mpl.style.use('ggplot')
sns.set(style='whitegrid')

In [62]:
df_KD = pd.read_csv('KD99_corrected.csv')
df_train = pd.read_csv('kd_train.csv')
df_test = pd.read_csv('kd_test.csv')

In [63]:
df_test.shape, df_train.shape

((49402, 43), (444619, 44))

In [64]:
df_KD.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [65]:
service_type = df_train.groupby('service').size().sort_values(ascending=False)

pd.reset_option("display.max_rows", "display.max_columns")

print(service_type)

service
ecr_i      253139
private     99930
http        57841
smtp         8756
other        6492
            ...  
urh_i          13
X11            11
tim_i           7
pm_dump         1
red_i           1
Length: 65, dtype: int64


In [66]:
for col in df_train.columns:
    print(col)

Unnamed: 0
duration
protocol_type
service
flag
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
lnum_compromised
lroot_shell
lsu_attempted
lnum_root
lnum_file_creations
lnum_shells
lnum_access_files
lnum_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
label
folds


In [67]:
df_train.describe()

,Unnamed: 0,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,folds
count,444619.000000,444619.000000,4.446190e+05,4.446190e+05,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,...,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000,444619.000000
mean,246998.385591,47.713051,3.175979e+03,8.573036e+02,0.000049,0.006468,0.000009,0.034067,0.000155,0.148221,...,188.614389,0.753592,0.030901,0.601631,0.006651,0.177094,0.176773,0.058021,0.057345,4.999991
std,142629.942725,708.552875,1.041486e+06,3.243744e+04,0.007034,0.135235,0.003674,0.776696,0.015800,0.355320,...,106.066729,0.410889,0.109221,0.481391,0.041892,0.380878,0.381196,0.230404,0.230026,2.581996
min,3.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,123403.500000,0.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,45.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,246992.000000,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,370577.500000,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
max,494021.000000,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,2.000000,30.000000,5.000000,1.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000


# Needs adding

In [68]:
del df_train['folds']

In [69]:
tftp_u_add = df_test.loc[df_test['service'] == 'tftp_u']
tftp_u_add

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
13529,140585,0,udp,tftp_u,SF,1,0,0,0,0,...,1,0.02,0.06,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [70]:
X11_add = df_train.loc[df_train['service'] == 'X11']
X11_add = X11_add.loc[X11_add['Unnamed: 0'] == 345610]

pm_dump_add = df_train.loc[df_train['service'] == 'pm_dump']

red_i_add = df_train.loc[df_train['service'] == 'red_i']

tim_i_add = df_train.loc[df_train['service'] == 'tim_i']
tim_i_add = tim_i_add.loc[tim_i_add['Unnamed: 0'] == 458739]

RSTOS0_add = df_train.loc[df_train['flag'] == 'RSTOS0']
RSTOS0_add = RSTOS0_add.loc[RSTOS0_add['Unnamed: 0'] == 42072]

In [71]:
df_test = pd.concat([df_test,X11_add,pm_dump_add,red_i_add,tim_i_add,RSTOS0_add])

In [72]:
df_train = pd.concat([df_train,tftp_u_add])

In [73]:
df_train.shape

(444620, 43)

In [74]:
df_test.shape

(49407, 43)

In [75]:
del df_test['Unnamed: 0']

df_test.index += 1
df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
1,0,tcp,private,S0,0,0,0,0,0,0,...,1,0.00,0.05,0.00,0.0,1.00,1.0,0.00,0.0,neptune.
2,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.00,0.0,0.00,0.0,smurf.
3,0,tcp,http,SF,186,21709,0,0,0,0,...,255,1.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,normal.
4,0,tcp,private,S0,0,0,0,0,0,0,...,18,0.07,0.07,0.00,0.0,1.00,1.0,0.00,0.0,neptune.
5,0,tcp,http,SF,341,6843,0,0,0,0,...,255,1.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14438,25,tcp,X11,SF,100788,32708,0,0,0,0,...,15,0.06,0.03,0.00,0.0,0.00,0.0,0.00,0.0,normal.
365903,0,tcp,pm_dump,SF,44,192,0,0,0,0,...,1,0.00,0.44,0.00,0.0,0.00,0.0,0.40,0.0,satan.
124851,0,icmp,red_i,SF,91,0,0,0,0,0,...,6,0.05,0.02,0.05,0.0,0.00,0.0,0.11,0.0,normal.
416566,0,icmp,tim_i,SF,564,0,0,0,0,0,...,2,1.00,0.00,1.00,0.0,0.00,0.0,0.00,0.0,pod.


In [76]:
del df_train['Unnamed: 0']

df_train.index += 1

In [77]:
df_train_resp = pd.DataFrame(df_train[['label']])
df_train = df_train.drop(labels='label', axis=1)


In [78]:
df_train_resp.head(), df_train_resp.shape

(      label
 1    smurf.
 2    smurf.
 3    smurf.
 4  neptune.
 5    smurf., (444620, 1))

In [79]:
df_test_resp = pd.DataFrame(df_test[['label']])
df_test = df_test.drop(labels='label', axis=1)

In [80]:
df_test_resp.head(), df_test_resp.shape

(      label
 1  neptune.
 2    smurf.
 3   normal.
 4  neptune.
 5   normal., (49407, 1))

In [81]:
for i in range(0,len(df_train_resp)):
    
    if df_train_resp.iat[i,0] == 'normal.':
        df_train_resp.iat[i,0] = '0'
    else:
        df_train_resp.iat[i,0] = '1'
df_train_resp.head()

,label
1,1
2,1
3,1
4,1
5,1


In [82]:
for i in range(0,len(df_test_resp)):
    
    if df_test_resp.iat[i,0] == 'normal.':
        df_test_resp.iat[i,0] = '0'
    else:
        df_test_resp.iat[i,0] = '1'
df_test_resp.head()

,label
1,1
2,1
3,0
4,1
5,0


In [83]:
df_train_resp = df_train_resp['label'].astype('int32')
df_train_resp.dtypes

dtype('int32')

In [84]:
df_test_resp = df_test_resp['label'].astype('int32')
df_test_resp.dtypes

dtype('int32')

In [85]:
df_train = pd.get_dummies(df_train)

In [86]:
df_test = pd.get_dummies(df_test)

In [87]:
df_train_resp.dtypes, df_test.dtypes

(dtype('int32'), duration          int64
 src_bytes         int64
 dst_bytes         int64
 land              int64
 wrong_fragment    int64
                   ...  
 flag_S1           uint8
 flag_S2           uint8
 flag_S3           uint8
 flag_SF           uint8
 flag_SH           uint8
 Length: 118, dtype: object)

In [88]:
df_train.shape, df_test.shape

((444620, 118), (49407, 118))

## Mutual Information

In [2]:
from sklearn.feature_selection import mutual_info_classif

In [90]:
data=mutual_info_classif(df_train, df_train_resp, discrete_features=False, random_state=1)

In [92]:
mutual_infos = pd.Series(data,index= df_train.columns)

In [93]:
mutual_infos.sort_values(ascending=False)

src_bytes                      0.455964
count                          0.452685
dst_bytes                      0.369601
dst_host_same_src_port_rate    0.301272
srv_count                      0.267770
                                 ...   
service_vmnet                  0.000000
service_whois                  0.000000
rerror_rate                    0.000000
service_daytime                0.000000
service_remote_job             0.000000
Length: 118, dtype: float64

## Random Forest

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer, matthews_corrcoef

In [34]:
MCC_scorer = make_scorer(matthews_corrcoef)

In [35]:
pipeline_rf = Pipeline([
    ('model', RandomForestClassifier(n_jobs=-1, random_state=1))
])

The random forest takes much longer to train on this fairly large dataset, so we don't actually do a hyperparameter grid search, only specifiying the number of estimators. We'll leave the grid search implemented in case we decide to try different hyperparameter values in the future.

In [36]:
param_grid_rf = {'model__n_estimators': [75]}

In [37]:
grid_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, scoring=MCC_scorer, n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, verbose=1, return_train_score=False)

In [38]:
for col in df_train.columns:
    print(col)

for col in df_test.columns:
    print(col)

duration
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
lnum_compromised
lroot_shell
lsu_attempted
lnum_root
lnum_file_creations
lnum_shells
lnum_access_files
lnum_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
protocol_type_icmp
protocol_type_tcp
protocol_type_udp
service_IRC
service_X11
service_Z39_50
service_auth
service_bgp
service_courier
service_csnet_ns
service_ctf
service_daytime
service_discard
service_domain
service_domain_u
service_echo
service_eco_i
service_ecr_i
service_efs
service_exec
service_finger
service_ftp
service_ftp_data
service_gopher
service_hostnames
service_http
service_http_443
service_imap4

In [39]:
grid_rf.fit(df_train, df_train_resp)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('model',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                    

In [40]:
grid_rf.best_score_

0.9994240914003581

In [41]:
grid_rf.best_params_

{'model__n_estimators': 75}

## Test Set Evaluation of the Best Model

In [4]:
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score

In [43]:
def classification_eval(estimator,df_test,df_test_resp):
    y_pred=estimator.predict(df_test)
    dec = np.int64(np.ceil(np.log10(len(df_test_resp))))
    
    print("Confusion Matrix")
    print(confusion_matrix(df_test_resp,y_pred),'\n')
    
    print('Classification report')
    print(classification_report(df_test_resp, y_pred, digits=dec))
    
    print("Scalar Metrics")
    format_str = '%%13s = %%.%if' % dec
    print(format_str % ('MCC', matthews_corrcoef(df_test_resp, y_pred)))
    
    if df_test_resp.nunique() <= 2: # Additional metrics for binary classification
        try:
            y_score = estimator.predict_proba(df_test)[:,1]
        except:
            y_score = estimator.decision_function(df_test)
        print(format_str % ('AUPRC', average_precision_score(df_test_resp, y_score)))
        print(format_str % ('AUROC', roc_auc_score(df_test_resp, y_score)))
    print(format_str % ("Cohen's kappa", cohen_kappa_score(df_test_resp, y_pred)))
    print(format_str % ('Accuracy', accuracy_score(df_test_resp, y_pred)))
    
    return y_pred

In [44]:
y_pred = classification_eval(grid_rf, df_test, df_test_resp)

Confusion Matrix
[[ 9731     0]
 [    3 39673]] 

Classification report
              precision    recall  f1-score   support

           0    0.99969   1.00000   0.99985      9731
           1    1.00000   0.99992   0.99996     39676

    accuracy                        0.99994     49407
   macro avg    0.99985   0.99996   0.99990     49407
weighted avg    0.99994   0.99994   0.99994     49407

Scalar Metrics
          MCC = 0.99981
        AUPRC = 1.00000
        AUROC = 1.00000
Cohen's kappa = 0.99981
     Accuracy = 0.99994


In [45]:
df_test.shape,df_train.shape,df_test_resp.shape,df_train_resp.shape

((49407, 118), (444620, 118), (49407,), (444620,))

In [46]:
df_test.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,520,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,186,21709,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,341,6843,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [47]:
df_train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
1,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [48]:
#for col in df_train.columns:
#print(col)

for col in df_test.columns:
    print(col)

duration
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
lnum_compromised
lroot_shell
lsu_attempted
lnum_root
lnum_file_creations
lnum_shells
lnum_access_files
lnum_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
protocol_type_icmp
protocol_type_tcp
protocol_type_udp
service_IRC
service_X11
service_Z39_50
service_auth
service_bgp
service_courier
service_csnet_ns
service_ctf
service_daytime
service_discard
service_domain
service_domain_u
service_echo
service_eco_i
service_ecr_i
service_efs
service_exec
service_finger
service_ftp
service_ftp_data
service_gopher
service_hostnames
service_http
service_http_443
service_imap4

We run the below code on the existing data set after it had been processed and we had used get_dummies.

In [49]:
traincols = []
testcols = []

In [50]:
for col in df_train.columns:
    traincols.append(col)
    
for col in df_test.columns:
    testcols.append(col)
    
print(traincols)
print(testcols)

['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'lnum_compromised', 'lroot_shell', 'lsu_attempted', 'lnum_root', 'lnum_file_creations', 'lnum_shells', 'lnum_access_files', 'lnum_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type_icmp', 'protocol_type_tcp', 'protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_e

In [51]:
for i in range(0,len(testcols)):
    
    if testcols[i] in traincols:
        pass
    else:
        print(testcols[i])

In [95]:
for i in range(0,len(traincols)):
    
    if traincols[i] in testcols:
        pass
    else:
        print(traincols[i])

After running the test on the model the first time, I was getting errors due to the features not matching. Because we perform get_dummies, we introduce exceptions where some services and flags don't exist within the test data set but do in the training data set and vice versa. 

As is outlined through the above analysis (if the steps in Data Cleansing 0 were not carried out, we are missing tftp_u from the training data and X11, pm_dump, red_i, tim_i and RSTOS0 from the test data. This is understandable as they occur very infrequently as was identified through earlier data analysis.

To resolve this issue without introducing events that didn't occur within the dataset, we will select events within the respective data sets to add to the events currently contained in each data set to resolve this issue, instead of creating new events which may disrupt the patterns identified.

After some data analysis, the following row from the test data was identified to contain the service factor: tftp_u.

## This needs to happen pretty much as soon as we import. After we remove folds from training data though.

In [53]:
#tftp_u_add = df_test.loc[df_test['service'] == 'tftp_u']
#print(tftp_u_add)

Additionally, the following rows are also required to adjust the testing data.

In [54]:
#X11_add = df_train.loc[df_train['service'] == 'X11']
#X11_add = X11_add.loc[X11_add['Unnamed: 0'] == 345610]

#pm_dump_add = df_train.loc[df_train['service'] == 'pm_dump']

#red_i_add = df_train.loc[df_train['service'] == 'red_i']

#tim_i_add = df_train.loc[df_train['service'] == 'tim_i']
#tim_i_add = tim_i_add.loc[tim_i_add['Unnamed: 0'] == 458739]

#RSTOS0_add = df_train.loc[df_train['flag'] == 'RSTOS0']
#RSTOS0_add = RSTOS0_add.loc[RSTOS0_add['Unnamed: 0'] == 42072]

In [55]:
#X11_add.shape
#df_test.loc[len(df_test)] = X11_add
#df_test.tail()

In [56]:
#df_test.shape

In [57]:
#df_train.shape

In [58]:
#df_test = pd.concat([df_test,X11_add,pm_dump_add,red_i_add,tim_i_add,RSTOS0_add])

In [98]:
results = pd.DataFrame(y_pred)
results

,0
0,1
1,1
2,0
3,1
4,0
...,...
49402,0
49403,1
49404,0
49405,1


In [110]:
a = pd.DataFrame(df_test_resp)
a = a.reset_index()
print(a)
a.iat[2,1]

        index  label
0           1      1
1           2      1
2           3      0
3           4      1
4           5      0
...       ...    ...
49402   14438      0
49403  365903      1
49404  124851      0
49405  416566      1
49406  257693      1

[49407 rows x 2 columns]


0

In [118]:
b = []
c = []

In [119]:
for i in range(0,len(a)):
    b.append(a.iat[i,1])

for i in range(0,len(results)):
    c.append(results.iat[i,0])

b,c

([1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,


In [122]:
for i in range(0,len(b)):
    if b[i] != c[i]:
            print(i)
    else:
        pass

6053
13076
47414


In [125]:
a.iat[6053,0]

6054

In [126]:
a.iat[13076,0]

13077

In [127]:
a.iat[47414,0]

47415

In [133]:
df_test.iloc[6053]

duration           127.0
src_bytes         1567.0
dst_bytes         2736.0
land                 0.0
wrong_fragment       0.0
                   ...  
flag_S1              0.0
flag_S2              0.0
flag_S3              0.0
flag_SF              1.0
flag_SH              0.0
Name: 6054, Length: 118, dtype: float64

In [134]:
q = pd.read_csv('kd_test.csv')

In [138]:
q.iloc[6054]

Unnamed: 0                     129998
duration                            0
protocol_type                    icmp
service                         ecr_i
flag                               SF
src_bytes                        1032
dst_bytes                           0
land                                0
wrong_fragment                      0
urgent                              0
hot                                 0
num_failed_logins                   0
logged_in                           0
lnum_compromised                    0
lroot_shell                         0
lsu_attempted                       0
lnum_root                           0
lnum_file_creations                 0
lnum_shells                         0
lnum_access_files                   0
lnum_outbound_cmds                  0
is_host_login                       0
is_guest_login                      0
count                             511
srv_count                         511
serror_rate                         0
srv_serror_r

In [142]:
q.iloc[47418]

Unnamed: 0                        71818
duration                              0
protocol_type                       tcp
service                         private
flag                                 S0
src_bytes                             0
dst_bytes                             0
land                                  0
wrong_fragment                        0
urgent                                0
hot                                   0
num_failed_logins                     0
logged_in                             0
lnum_compromised                      0
lroot_shell                           0
lsu_attempted                         0
lnum_root                             0
lnum_file_creations                   0
lnum_shells                           0
lnum_access_files                     0
lnum_outbound_cmds                    0
is_host_login                         0
is_guest_login                        0
count                               220
srv_count                            11


In [143]:
q.iloc[13077]

Unnamed: 0                     180961
duration                            0
protocol_type                    icmp
service                         ecr_i
flag                               SF
src_bytes                        1032
dst_bytes                           0
land                                0
wrong_fragment                      0
urgent                              0
hot                                 0
num_failed_logins                   0
logged_in                           0
lnum_compromised                    0
lroot_shell                         0
lsu_attempted                       0
lnum_root                           0
lnum_file_creations                 0
lnum_shells                         0
lnum_access_files                   0
lnum_outbound_cmds                  0
is_host_login                       0
is_guest_login                      0
count                             511
srv_count                         511
serror_rate                         0
srv_serror_r

In [145]:
z = q.groupby('label')

In [149]:
pd.options.display.max_columns = 999

In [150]:
z.describe()

Unnamed: 0                                          \
                      count           mean            std       min   
label                                                                 
back.                 214.0   74957.098131   97733.268821   39738.0   
buffer_overflow.        2.0  485176.000000       1.414214  485175.0   
guess_passwd.           3.0   22771.333333      20.526406   22754.0   
imap.                   1.0   76318.000000            NaN   76318.0   
ipsweep.              147.0  155700.108844  155746.364808   26509.0   
neptune.            10603.0  284727.931057  157777.600960   53604.0   
nmap.                  22.0   54343.954545   15090.900309   43068.0   
normal.              9729.0  160512.764107  173341.822794       1.0   
pod.                   28.0  257664.285714  144966.123941   15794.0   
portsweep.            103.0  244231.262136  147894.843801   22823.0   
satan.                153.0  126610.418301   25869.917058   41728.0   
smurf.              28200.0  265786.873404  107469.239815    7796.0   
teardrop.              92.0  268631.326087  152970.372148   19303.0   
warezclient.          103.0  143764.728155    2907.520678  140033.0   
warezmaster.            2.0   77157.000000       2.828427   77155.0   

                                                           duration  \
                        25%       50%        75%       max    count   
label                                                                 
back.              40287.25   51109.0   51748.75  480553.0    214.0   
buffer_overflow.  485175.50  485176.0  485176.50  485177.0      2.0   
guess_passwd.      22760.00   22766.0   22780.00   22794.0      3.0   
imap.              76318.00   76318.0   76318.00   76318.0      1.0   
ipsweep.           52228.50   52538.0  344670.00  485173.0    147.0   
neptune.          114923.50  361928.0  391283.50  486880.0  10603.0   
nmap.              43103.75   43159.0   75271.00   76081.0     22.0   
normal.            28787.00   83651.0  344603.00  494009.0   9729.0   
pod.              141627.75  344913.0  371419.25  456269.0     28.0   
portsweep.         86306.00  144845.0  370006.50  455929.0    103.0   
satan.             92053.00  140862.0  149518.00  149853.0    153.0   
smurf.            189962.75  260647.0  330345.50  490863.0  28200.0   
teardrop.         141541.75  370091.5  370312.25  490948.0     92.0   
warezclient.      141290.50  142614.0  147268.00  148947.0    103.0   
warezmaster.       77156.00   77157.0   77158.00   77159.0      2.0   

                                                                               \
                        mean          std   min    25%   50%     75%      max   
label                                                                           
back.               0.261682     1.462113   0.0   0.00   0.0    0.00     14.0   
buffer_overflow.   80.000000    66.468037  33.0  56.50  80.0  103.50    127.0   
guess_passwd.       0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
imap.              41.000000          NaN  41.0  41.00  41.0   41.00     41.0   
ipsweep.            0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
neptune.            0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
nmap.               0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
normal.           237.337445  1466.530091   0.0   0.00   0.0    0.00  30190.0   
pod.                0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
portsweep.        931.640777  4519.995666   0.0   0.00   0.0    0.00  30518.0   
satan.              0.124183     1.096167   0.0   0.00   0.0    0.00     11.0   
smurf.              0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
teardrop.           0.000000     0.000000   0.0   0.00   0.0    0.00      0.0   
warezclient.      809.165049  2528.408184   0.0   0.00   0.0    1.00  15160.0   
warezmaster.        9.500000     0.707107   9.0   9.25   9.5    9.75     10.0   



We need to perform similar data cleansing for the data that will be used to test out of sample predictions for our teardrop data.

In [5]:
df_teardrop = pd.read_csv('KD99_teardrop.csv')
df_noteardrop = pd.read_csv('KD99_noteardrop.csv')

In [6]:
df_teardrop.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,udp,private,SF,28,0,0,1,0,0,...,1,0.01,0.05,0.01,0.0,0.0,0.0,0.0,0.0,teardrop.
1,0,udp,private,SF,28,0,0,3,0,0,...,2,0.03,0.05,0.03,0.0,0.0,0.0,0.0,0.0,teardrop.
2,0,udp,private,SF,28,0,0,3,0,0,...,3,0.04,0.05,0.04,0.0,0.0,0.0,0.0,0.0,teardrop.
3,0,udp,private,SF,28,0,0,3,0,0,...,4,0.05,0.05,0.05,0.0,0.0,0.0,0.0,0.0,teardrop.
4,0,udp,private,SF,28,0,0,3,0,0,...,5,0.06,0.05,0.06,0.0,0.0,0.0,0.0,0.0,teardrop.


In [7]:
df_noteardrop.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


We need to seperate out the response variables and the features to ensure we can use the later models so we will do this here.

In [8]:
df_noteardrop_resp = pd.DataFrame(df_noteardrop[['label']])
df_noteardrop = df_noteardrop.drop(labels='label', axis=1)
df_noteardrop_resp.head()

,label
0,normal.
1,normal.
2,normal.
3,normal.
4,normal.


We need to do the same splitting for the teardrop data.

In [9]:
df_teardrop_resp = pd.DataFrame(df_teardrop[['label']])
df_teardrop = df_teardrop.drop(labels='label', axis=1)
df_teardrop_resp.head()

,label
0,teardrop.
1,teardrop.
2,teardrop.
3,teardrop.
4,teardrop.


To allow us to use the data in random forest setting, we need to reclassify the response data. In this case, we need to split the data into normal and non-normal processes and represent them using integers. We will do this for both data sets below.

In [11]:
def labeler(df):
    for i in range(0,len(df)):
    
        if df.iat[i,0] == 'normal.':
            df.iat[i,0] = '0'
        else:
            df.iat[i,0] = '1'

In [12]:
labeler(df_teardrop_resp)
labeler(df_noteardrop_resp)

In [13]:
df_noteardrop = pd.get_dummies(df_noteardrop)
df_teardrop = pd.get_dummies(df_teardrop)

df_teardrop_resp = df_teardrop_resp['label'].astype('int32')
df_noteardrop_resp = df_noteardrop_resp['label'].astype('int32')

In [17]:
df_noteardrop_resp.dtypes, df_noteardrop.dtypes, df_teardrop_resp.dtypes, df_teardrop.dtypes

(dtype('int32'), duration          int64
 src_bytes         int64
 dst_bytes         int64
 land              int64
 wrong_fragment    int64
                   ...  
 flag_S1           uint8
 flag_S2           uint8
 flag_S3           uint8
 flag_SF           uint8
 flag_SH           uint8
 Length: 118, dtype: object, dtype('int32'), duration                         int64
 src_bytes                        int64
 dst_bytes                        int64
 land                             int64
 wrong_fragment                   int64
 urgent                           int64
 hot                              int64
 num_failed_logins                int64
 logged_in                        int64
 lnum_compromised                 int64
 lroot_shell                      int64
 lsu_attempted                    int64
 lnum_root                        int64
 lnum_file_creations              int64
 lnum_shells                      int64
 lnum_access_files                int64
 lnum_outbound_cmds      

We can see from the above that this process has worked correctly and we now have all factors in terms of integers (using dummy variables).

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer, matthews_corrcoef

In [20]:
MCC_scorer = make_scorer(matthews_corrcoef)

In [21]:
pipeline_rf = Pipeline([
    ('model', RandomForestClassifier(n_jobs=-1, random_state=1))
])

The random forest takes much longer to train on this fairly large dataset, so we don't actually do a hyperparameter grid search, only specifiying the number of estimators. We'll leave the grid search implemented in case we decide to try different hyperparameter values in the future.

In [25]:
param_grid_rf_tear = {'model__n_estimators': [75]}

In [26]:
grid_rf_tear = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf_tear, scoring=MCC_scorer, n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, verbose=1, return_train_score=False)

In [28]:
grid_rf_tear.fit(df_noteardrop, df_noteardrop_resp)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('model',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                    

In [30]:
grid_rf_tear.best_score_

0.9824006591439348

In [31]:
grid_rf_tear.best_params_

{'model__n_estimators': 75}

## Test Set Evaluation of the Model

In [86]:
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score

In [89]:
def classification_eval(estimator,df_teardrop,df_teardrop_resp):
    y_pred_tear = estimator.predict(df_teardrop)
    dec = np.int64(np.ceil(np.log10(len(df_teardrop_resp))))
    
    print("Confusion Matrix")
    print(confusion_matrix(df_teardrop_resp,y_pred_tear),'\n')
    
    print('Classification report')
    print(classification_report(df_teardrop_resp, y_pred_tear, digits=dec))
    
    print("Scalar Metrics")
    format_str = '%%13s = %%.%if' % dec
    print(format_str % ('MCC', matthews_corrcoef(df_teardrop_resp, y_pred_tear)))
    
    if df_teardrop_resp.nunique() <= 2: # Additional metrics for binary classification
        try:
            y_score_tear = estimator.predict_proba(df_teardrop)[:,1]
        except:
            y_score_tear = estimator.decision_function(df_teardrop)
        print(format_str % ('AUPRC', average_precision_score(df_teardrop_resp, y_score_tear)))
        #print(format_str % ('AUROC', roc_auc_score(df_teardrop_resp, y_score_tear)))
    print(format_str % ("Cohen's kappa", cohen_kappa_score(df_teardrop_resp, y_pred_tear)))
    print(format_str % ('Accuracy', accuracy_score(df_teardrop_resp, y_pred_tear)))

In [90]:
classification_eval(grid_rf_tear, df_teardrop, df_teardrop_resp)

Confusion Matrix
[[  0   0]
 [788 191]] 

Classification report
              precision    recall  f1-score   support

           0      0.000     0.000     0.000         0
           1      1.000     0.195     0.326       979

    accuracy                          0.195       979
   macro avg      0.500     0.098     0.163       979
weighted avg      1.000     0.195     0.326       979

Scalar Metrics
          MCC = 0.000
        AUPRC = 1.000
Cohen's kappa = 0.000
     Accuracy = 0.195


/Users/User-MC/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/User-MC/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [99]:
predictions_teardrop = pd.DataFrame(grid_rf_tear.predict(df_teardrop))
predictions_teardrop.head()

,0
0,0
1,0
2,0
3,0
4,0


In [100]:
predictions_teardrop.loc[-1] = ['0'] # grabbing the current header and setting it as a row
predictions_teardrop.index = predictions_teardrop.index + 2 # resetting the index
predictions_teardrop.sort_index(inplace=True) # ensuring index is sorted

# adding column names
predictions_teardrop.columns = ['label']
predictions_teardrop.head()

,label
1,0
2,0
3,0
4,0
5,0


In [101]:
predictions_teardrop.groupby('label').size().sort_values(ascending=False)

label
0    788
1    191
0      1
dtype: int64

In [36]:
notearcols = []
tearcols = []

In [37]:
for col in df_noteardrop.columns:
    notearcols.append(col)
    
for col in df_teardrop.columns:
    tearcols.append(col)

In [64]:
misscols = []

In [65]:
for i in range(0,len(notearcols)):
    
    if notearcols[i] in tearcols:
        pass
    else:
        misscols.append(notearcols[i])

print(misscols)

['protocol_type_icmp', 'protocol_type_tcp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_hostnames', 'service_http', 'service_http_443', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_red_i', 'service_remote_job', 'service_rje', 'service_shell', 'service_smtp', 'service_sql_net', 'service_ssh', 'service_sunrpc', 'service_supdup', 'se

In [68]:
len(misscols)

77

In [41]:
df_noteardrop.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In this case, what we need to do is add those columns to the testing data set and have 0's in every event since they never occur in the test date. This allows us to have the same number of features and shouldn't alter the results at all (although I'm not 100% sure on this).

In [59]:
testdatafr = pd.DataFrame(columns=['a','b'])
testdatafr.loc[0] = ['1','2']
testdatafr.loc[1] = ['3','4']

In [60]:
testdatafr.head()

,a,b
0,1,2
1,3,4


In [61]:
testdatafr['e'] = '0'

In [62]:
testdatafr.head()

,a,b,e
0,1,2,0
1,3,4,0


In [66]:
testdatafr.shape

(2, 3)

In [69]:
for i in range(0,len(misscols)):
    testdatafr[misscols[i]] = '0'

In [70]:
testdatafr.shape

(2, 80)

In [71]:
testdatafr.head()

,a,b,e,protocol_type_icmp,protocol_type_tcp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,...,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SH
0,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
testdata = df_teardrop

In [84]:
testdata.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SH
0,0,28,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
testdata.shape

(979, 118)

In [82]:
for col in testdata.columns:
    print(col)

duration
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
lnum_compromised
lroot_shell
lsu_attempted
lnum_root
lnum_file_creations
lnum_shells
lnum_access_files
lnum_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
protocol_type_udp
service_private
flag_SF
protocol_type_icmp
protocol_type_tcp
service_IRC
service_X11
service_Z39_50
service_auth
service_bgp
service_courier
service_csnet_ns
service_ctf
service_daytime
service_discard
service_domain
service_domain_u
service_echo
service_eco_i
service_ecr_i
service_efs
service_exec
service_finger
service_ftp
service_ftp_data
service_gopher
service_hostnames
service_http
servic

In [78]:
for i in range(0,len(misscols)):
    testdata[misscols[i]] = '0'